<a href="https://colab.research.google.com/github/mgmeti/Data-Analysis/blob/main/Pet_Ownership_Analysis_in_Southfield.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd


In [4]:
# Load 'owners.csv' into a DataFrame
owners_df = pd.read_csv('owners.csv')

# Load 'pets.csv' into a DataFrame
pets_df = pd.read_csv('pets.csv')

# Load 'procedure_details.csv' into a DataFrame
procedure_details_df = pd.read_csv('procedure_details.csv')

# Load 'procedures.csv' into a DataFrame
procedures_df = pd.read_csv('procedures.csv')


In [5]:
# Display the first few rows of each DataFrame
print("Owners DataFrame:")
print(owners_df.head())

print("\nPets DataFrame:")
print(pets_df.head())

print("\nProcedure Details DataFrame:")
print(procedure_details_df.head())

print("\nProcedures DataFrame:")
print(procedures_df.head())


Owners DataFrame:
   OwnerID     Name     Surname          StreetAddress                 City  \
0     6049   Debbie    Metivier        315 Goff Avenue         Grand Rapids   
1     2863     John   Sebastian      3221 Perry Street              Davison   
2     3518   Connie      Pauley  1539 Cunningham Court  Bloomfield Township   
3     3663     Lena  Haliburton   4217 Twin Oaks Drive        Traverse City   
4     1070  Jessica   Velazquez   3861 Woodbridge Lane           Southfield   

  State StateFull  ZipCode  
0    MI  Michigan    49503  
1    MI  Michigan    48423  
2    MI  Michigan    48302  
3    MI  Michigan    49684  
4    MI  Michigan    48034  

Pets DataFrame:
     PetID     Name    Kind  Gender  Age  OwnerID
0  J6-8562   Brandy     Dog    male   11     5168
1  Q0-2001   Roomba     Cat    male    9     5508
2  M0-2904    Simba     Cat    male    1     3086
3  R3-7551   Keller  Parrot  female    2     7908
4  P2-7342  Cuddles     Dog    male   13     4378

Procedure Detai

Q1 ) What is the name of the oldest dog in Southfield?

Step 1: Merge the Pets DataFrame with the Owners DataFrame based on the common column OwnerID.

Step 2: Filter the rows where the Kind is 'Dog' and City is 'Southfield'.

Step 3: Find the dog with the maximum age.

Step 4: Get the name of the oldest dog.

In [6]:
# Step 1: Merge the Pets DataFrame with the Owners DataFrame
merged_df = pd.merge(pets_df, owners_df, on='OwnerID')

# Step 2: Filter rows for dogs in Southfield
oldest_dog_southfield = merged_df[(merged_df['Kind'] == 'Dog') & (merged_df['City'] == 'Southfield')]

# Step 3: Find the dog with the maximum age
oldest_dog = oldest_dog_southfield[oldest_dog_southfield['Age'] == oldest_dog_southfield['Age'].max()]


# Step 4: Get the name of the oldest dog
oldest_dog_name = oldest_dog['Name_x'].values[0]

print("The name of the oldest dog in Southfield is:", oldest_dog_name)


The name of the oldest dog in Southfield is: Crockett


Q2 ) What is the average (mean) number of pets per city?



1. Merge the 'Owners' DataFrame with the 'Pets' DataFrame using the common column'OwnerID' as the key to associate each pet with its owner and their city.
2. Group the merged DataFrame by 'City'.
3. Calculate the average number of pets in each city.





In [7]:
# Step 1: Merge 'Owners' DataFrame with 'Pets' DataFrame
merged_df = pd.merge(owners_df, pets_df, on='OwnerID')

# Step 2: Group by 'City' and count the number of pets in each city
average_pets_per_city = merged_df.groupby('City')['PetID'].count()

# Step 3: Calculate the mean (average) number of pets per city
mean_pets_per_city = average_pets_per_city.mean()

print("Average number of pets per city:", mean_pets_per_city)


Average number of pets per city: 2.5


Q3 ) Which owner spent the most on procedures for their pet(s)?

Q4 ) How many owners spent 20 dollars or more on procedures for their pets?

In [8]:
# Merge the 'procedures_df' with 'procedure_details_df' to get the price of each procedure
merged_procedures_df = pd.merge(procedures_df, procedure_details_df, on=['ProcedureType', 'ProcedureSubCode'])

# Merge the 'pets_df' with 'merged_procedures_df' to get the price of each procedure for each pet
merged_df = pd.merge(pets_df, merged_procedures_df, on='PetID')

# Calculate the total cost of procedures for each pet
merged_df['TotalProcedureCost'] = merged_df['Price']

# Group by 'OwnerID' and calculate the total procedure cost for each owner
owner_procedure_costs = merged_df.groupby('OwnerID')['TotalProcedureCost'].sum()

# Find the OwnerID with the maximum total procedure cost
max_spending_owner_id = owner_procedure_costs.idxmax()

# Retrieve the owner's details from the 'owners_df' using the found OwnerID
max_spending_owner_details = owners_df.loc[owners_df['OwnerID'] == max_spending_owner_id]

# Print the result
print("Owner who spent the most on procedures:")
print(max_spending_owner_details[['Name', 'Surname', 'City', 'State']])


# Filter the merged DataFrame to include only those owners who spent $20 or more on procedures
owners_spent_20_or_more = merged_df.groupby('OwnerID')['TotalProcedureCost'].sum() >= 20

# Count the number of owners who meet the condition (spent $20 or more)
num_owners_spent_20_or_more = owners_spent_20_or_more.sum()

# Print the result
print("Number of owners who spent $20 or more on procedures for their pets:", num_owners_spent_20_or_more)



Owner who spent the most on procedures:
      Name Surname          City State
70  Daniel     Fay  Grand Rapids    MI
Number of owners who spent $20 or more on procedures for their pets: 12


Q5 ) How many owners have at least two different kinds of pets (e.g., a dog and a cat)?

In [9]:
# Step 1: Merge the DataFrames
merged_df = pd.merge(owners_df, pets_df, on='OwnerID', how='inner')

# Step 2: Group and Filter Data
owners_with_multiple_pets = merged_df.groupby('OwnerID')['Kind'].nunique() >= 2

# Step 3: Count the Owners
num_owners_with_multiple_pets = owners_with_multiple_pets.sum()

# Step 4: Calculate and Print the Result
print("Number of owners with at least two different kinds of pets:", num_owners_with_multiple_pets)


Number of owners with at least two different kinds of pets: 4


Q6 ) How many owners have pets where the first letter of their name (OwnerName) matches the first letter of their pet's name (PetName)? E.g., Cookie and Charles.

In [10]:
# Step 1: Merge the 'owners_df' and 'pets_df' DataFrames on 'OwnerID' and 'OwnerID' columns
merged_df = owners_df.merge(pets_df, left_on='OwnerID', right_on='OwnerID', suffixes=('_owner', '_pet'))

# Step 2: Create a new column to store whether the first letter of the owner's name matches the first letter of the pet's name
merged_df['NameMatch'] = merged_df.apply(lambda row: row['Name_owner'][0] == row['Name_pet'][0], axis=1)

# Step 3: Count the number of occurrences where the first letter matches
matching_owners_count = merged_df[merged_df['NameMatch']].shape[0]

print("Number of owners with pets where the first letter of their name matches the first letter of their pet's name:", matching_owners_count)


Number of owners with pets where the first letter of their name matches the first letter of their pet's name: 8


Q7 )  What percentage of pets received a vaccination?

In [19]:
# Count the total number of pets that received a vaccination
vaccinated_pets_count = (procedure_details_df['ProcedureType'] == 'VACCINATIONS').sum()

# Calculate the total number of pets in the dataset
total_pets_count = len(procedure_details_df)

# Calculate the percentage of pets that received a vaccination
percentage_vaccinated_pets = (vaccinated_pets_count / total_pets_count) * 100

print("Percentage of pets that received a vaccination: {:.2f}%".format(percentage_vaccinated_pets))

Percentage of pets that received a vaccination: 14.63%


Q8 )  What percentage of cities have more male pets than female pets?

In [12]:
# Merge owners_df and pets_df on OwnerID
merged_df = pd.merge(owners_df, pets_df, on='OwnerID')

# Group by City and Gender, and calculate the count of each gender in each city
gender_count_by_city = merged_df.groupby(['City', 'Gender']).size().unstack(fill_value=0)

# Calculate the percentage of cities with more male pets than female pets
cities_with_more_male_pets = gender_count_by_city[gender_count_by_city['male'] > gender_count_by_city['female']]
percentage_cities_more_male_pets = (len(cities_with_more_male_pets) / len(merged_df['City'].unique())) * 100

print(f"Percentage of cities with more male pets than female pets: {percentage_cities_more_male_pets:.2f}%")

Percentage of cities with more male pets than female pets: 57.50%


Q9 )  Which city's pet sample is made up of exactly 70% dogs?

In [13]:
# Step 1: Merge the Pets DataFrame with the Owners DataFrame
merged_df = pd.merge(pets_df, owners_df, on='OwnerID')

# Step 2: Calculate the total number of pets and the number of dogs for each city
city_pets_count = merged_df.groupby('City')['Kind'].value_counts().unstack().fillna(0)
city_pets_count['Total'] = city_pets_count.sum(axis=1)
city_pets_count['Percentage_Dogs'] = (city_pets_count['Dog'] / city_pets_count['Total']) * 100

# Step 4: Identify the city where the percentage of dogs is exactly 70%
city_with_70_percent_dogs = city_pets_count[city_pets_count['Percentage_Dogs'] == 70.0]

print("City's pet sample with exactly 70% dogs:")
print(city_with_70_percent_dogs.index[0])


City's pet sample with exactly 70% dogs:
Grand Rapids


In [ ]:
# Step 1: Merge the procedures_df and procedure_details_df DataFrames
merged_df = pd.merge(procedures_df, procedure_details_df, on=['ProcedureType', 'ProcedureSubCode'])

print(merged_df)
# Step 2: Filter the merged DataFrame to include only rows where the ProcedureType is 'VACCINATIONS'
vaccinations_df = merged_df[merged_df['ProcedureType'] == 'VACCINATIONS']

# Step 3: Count the total number of vaccinations performed
total_vaccinations_count = len(vaccinations_df)

# Step 4: Calculate the total number of pets in the procedures_df DataFrame
total_pets_count = len(procedures_df)

# Step 5: Calculate the percentage of pets that received a vaccination
percentage_vaccinated_pets = (total_vaccinations_count / total_pets_count) * 100

print("Percentage of pets that received a vaccination: {:.2f}%".format(percentage_vaccinated_pets))

In [23]:
# Count the occurrences of each unique value in the 'ProcedureType' column
procedure_type_counts = merged_df['ProcedureType'].value_counts()

print("Count of each ProcedureType:")
print(procedure_type_counts)

Count of each ProcedureType:
VACCINATIONS         29
ORTHOPEDIC            4
GROOMING              4
GENERAL SURGERIES     4
Name: ProcedureType, dtype: int64


In [25]:
# Count the occurrences of each unique value in the 'ProcedureType' column
procedure_type_counts = procedure_details_df['ProcedureType'].value_counts()

print("Count of each ProcedureType:")
print(procedure_type_counts)

Count of each ProcedureType:
GENERAL SURGERIES    19
ORTHOPEDIC            9
VACCINATIONS          6
OFFICE FEES           3
GROOMING              3
HOSPITALIZATION       1
Name: ProcedureType, dtype: int64
